# Проект по SQL

В рамках работы предстоит проанализировать данные крупного сервиса для чтения книг по подписке. Основная задача — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных:** 

**Таблица "books"**
Содержит данные о книгах:
book_id — идентификатор книги;
author_id — идентификатор автора;
title — название книги;
num_pages — количество страниц;
publication_date — дата публикации книги;
publisher_id — идентификатор издателя.

**Таблица "authors"**
Содержит данные об авторах:
author_id — идентификатор автора;
author — имя автора.

**Таблица "publishers"**
Содержит данные об издательствах:
publisher_id — идентификатор издательства;
publisher — название издательства;

**Таблица "ratings"**
Содержит данные о пользовательских оценках книг:
rating_id — идентификатор оценки;
book_id — идентификатор книги;
username — имя пользователя, оставившего оценку;
rating — оценка книги.

**Таблица "reviews"**
Содержит данные о пользовательских обзорах:
review_id — идентификатор обзора;
book_id — идентификатор книги;
username — имя автора обзора;
text — текст обзора.

# Основная часть

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine


In [3]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [4]:
# создадим функцию для вывода запроов
def result(query):
    con=engine.connect()
    f = pd.io.sql.read_sql(sql=text(query), con = con)
    return f

Выведем исходные данные

In [5]:
# по таблице с книгами
q = '''SELECT * FROM books LIMIT 5'''
result(q)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
q = '''SELECT * FROM authors LIMIT 5'''
result(q)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
q = '''SELECT * FROM publishers LIMIT 5'''
result(q)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
q = '''SELECT * FROM ratings LIMIT 5'''
result(q)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
q = '''SELECT * FROM reviews LIMIT 5'''
result(q)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Посчитаем, сколько книг вышло после 1 января 2000 года:**

In [10]:
q = '''SELECT 
            COUNT(book_id)
            FROM books
            WHERE  EXTRACT (year from cast(publication_date as timestamp)) >= 2000'''

result(q)

,count
0,821


Всего выпущена 821 книга после 1 января 2000 года

**Для каждой книги посчитаем количество обзоров и среднюю оценку**

In [11]:
q = ''' SELECT
                books.title AS title,
                books.book_id AS book_id,
                COUNT(DISTINCT reviews.review_id) AS count_reviews, 
                AVG(ratings.rating) AS avg_rating
                FROM books
            INNER JOIN ratings ON ratings.book_id = books.book_id
            LEFT JOIN reviews ON reviews.book_id = books.book_id
            GROUP BY
                books.title,
                books.book_id
            ORDER BY
                count_reviews DESC'''

result(q)

,title,book_id,count_reviews,avg_rating
0,Twilight (Twilight #1),948,7,3.662500
1,Water for Elephants,963,6,3.977273
2,The Glass Castle,734,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,695,6,4.081081
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.666667
996,The Natural Way to Draw,808,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,0,5.000000
998,Essential Tales and Poems,221,0,4.000000


Наибольшее количество обзоров наблюдается у книги "Twilight (Twilight #1)" - 7	

**Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры**

In [12]:
q = '''  SELECT 
                publishers.publisher_id, 
                publishers.publisher, 
                COUNT(books.book_id) as books_count
            FROM publishers
            JOIN books ON publishers.publisher_id = books.publisher_id
            WHERE 
                books.num_pages > 50
            GROUP BY publishers.publisher_id
            ORDER BY books_count DESC
            LIMIT 5'''

result(q)

,publisher_id,publisher,books_count
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,35,Bantam,19


Издательство "Penguin Books" выпустило 42 книги, с количеством страниц более 50, это самый высокий показатель среди других издательств.

**Определим автора с самой высокой средней оценкой книг — учитем только книги с 50 и более оценками**

In [13]:
q = ''' SELECT
                authors.author_id AS id,
                authors.author AS authors,
                AVG(ratings.rating) AS avg,
                COUNT(ratings.rating) AS count
            FROM authors
            INNER JOIN books ON books.author_id = authors.author_id
            INNER JOIN ratings ON ratings.book_id = books.book_id
            GROUP BY
                id,
                author
            HAVING 
                COUNT(ratings.rating) >= 50                            
            ORDER BY
            count DESC LIMIT 5
                        '''

result(q)

,id,authors,avg,count
0,236,J.K. Rowling/Mary GrandPré,4.288462,312
1,240,J.R.R. Tolkien,4.240964,166
2,554,Stephenie Meyer,3.662500,160
3,106,Dan Brown,3.741259,143
4,445,Nicholas Sparks,3.882883,111


Автором с самой высокой средней оценкой является J.K. Rowling/Mary GrandPré - 4.29, учитывая только 50 и более книг.

**Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок**

In [14]:
q = ''' SELECT
            AVG(Sub.n_reviews)
            FROM
        (SELECT
            username,
            COUNT(review_id) AS n_reviews
        FROM reviews
        WHERE 
            username IN (
                    SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING 
                        COUNT(rating_id) > 48)
        GROUP BY username
        ) AS Sub
                        '''

result(q)

,avg
0,24.0


Среднее количество обзоров для пользователей, которые поставили более 48 оценок, составляет 24

# Заключение

В результате анализа данных из базы получены следующие выводы:
1. Всего выпущена 821 книга после 1 января 2000 года;
2. Наибольшее количество обзоров наблюдается у книги "Twilight (Twilight #1)" - 7;
3. Издательство "Penguin Books" выпустило 42 книги, с количеством страниц более 50, это самый высокий показатель среди других издательств;
4. Автором с самой высокой средней оценкой является J.K. Rowling/Mary GrandPré - 4.29, учитывая только 50 и более книг;
5. Среднее количество обзоров для пользователей, которые поставили более 48 оценок, составляет 24